In [4]:
# !pip install gensim

In [3]:
# !pip install python-Levenshtein

In [16]:
# 
import gensim
# 


C:\Users\Asim Ramzan\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
import pandas as pd


In [14]:
df = pd.read_json('Cell_Phones_and_Accessories_5.json', lines = True)

In [15]:
df.sample(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
64517,A3BH0G750NO10Q,B005ZG0342,"Jump Squad ""GKR""","[0, 0]",The product is excellent. I've dropped the ph...,5,Easy to use and works great,1349913600,"10 11, 2012"
166900,A1R4FLMBTTOZP2,B00DCIZJ5S,"Monu Koshy ""Monu Koshy""","[0, 0]","Does add a good amount of protection, but does...",3,Looks alright.,1385424000,"11 26, 2013"
51098,A1HY300EZK9A8Y,B005DOTWSY,cheryl peeler,"[0, 0]","Love to sparkle, just what I needed! My only p...",4,Nice and shiny!,1354665600,"12 5, 2012"
23386,A1DM4UO4CRFNQV,B003YDZXV6,"Samuel D. Snead ""Analytical Shopper""","[0, 0]",This battery was better than my previous one. ...,2,Decent Battery,1363392000,"03 16, 2013"
137650,A2Y8K8FC7QBMQC,B00A70MOIO,"K. K. Ying ""K.K. YING""","[0, 0]",Good product; you got what you paid; but if th...,4,Good Product,1395619200,"03 24, 2014"


In [17]:
df.shape

(194439, 9)

In [20]:
# using only the Review text
df.reviewText[1]

'These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :)'

In [22]:
# removing the punctuation and capital letters etc so that it could be easy to process
# we will use gensim library to generate best result
gensim.utils.simple_preprocess('These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :)')

# we can see all letters in lower case and commas and punctuation , useless words are like i, the, an ,is are removed

['these',
 'stickers',
 'work',
 'like',
 'the',
 'review',
 'says',
 'they',
 'do',
 'they',
 'stick',
 'on',
 'great',
 'and',
 'they',
 'stay',
 'on',
 'the',
 'phone',
 'they',
 'are',
 'super',
 'stylish',
 'and',
 'can',
 'share',
 'them',
 'with',
 'my',
 'sister']

In [26]:
# lets convert whole columns into panda series objects
reviewText = df.reviewText.apply(gensim.utils.simple_preprocess)
reviewText

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [28]:
# Initializing gensim model , gensim is NLP library builtin with word2vector class
model = gensim.models.Word2Vec(
 
    window = 10, # windows = 10  words before and after the target word
    min_count = 2,
    workers = 4,
    
)

In [30]:
# building the vocabulary 
model.build_vocab(reviewText, progress_per= 1000,  )

In [32]:
model.corpus_count

194439

In [33]:
model.epochs

5

In [35]:
model.train(reviewText, total_examples = model.corpus_count, epochs = model.epochs)


(61504302, 83868975)

In [36]:
model.save('./word2vec-amazon-Cell_Phones_and_Accessories_Reviews.model')

In [38]:
model.wv.most_similar('bad')

[('terrible', 0.6774733066558838),
 ('shabby', 0.6670687794685364),
 ('good', 0.5975676774978638),
 ('horrible', 0.5727895498275757),
 ('okay', 0.5398114323616028),
 ('disappointing', 0.5329607129096985),
 ('crappy', 0.5280848145484924),
 ('awful', 0.5210759043693542),
 ('sad', 0.5028854608535767),
 ('poor', 0.5005925893783569)]

In [45]:
model.wv.similarity(w1 = 'good', w2 = 'great')
#  high similarity

0.77627814

In [46]:
model.wv.similarity(w1 = 'good', w2 = 'product')

# v low similarity

-0.05308377

In [47]:
model.wv.similarity(w1 = 'good', w2 = 'cool')


0.60239935